**[← Back to Course Overview](https://github.com/buildLittleWorlds/gateway-to-densworld)**

# Tutorial 11: The Living Ledger
## Exploring Densworld's Event History

---

*In the Capital Archives, beyond the manuscript rooms and expedition records, there is a special chamber called the Temporal Registry.*

*Here, archivists maintain the Living Ledger—a record of everything that has happened in Densworld. Not just what exists, but when things happened, why they happened, and what consequences followed.*

*Chief Archivist Mink calls it "the story of cause and effect."*

*"Your creatures.csv tells you the Maw Beast exists," Mink explains. "The Living Ledger tells you when it was first encountered, who discovered it, and what happened to them afterward."*

*Today, you gain access to the Ledger.*

---

## What You'll Learn

By the end of this tutorial, you will:

1. **Load** the event log using a special Python module
2. **Query** events by type, actor, date, and location
3. **Trace** causal chains (what caused what)
4. **Connect** events to the regions you explored in earlier tutorials

This tutorial brings together everything you've learned:
- Python basics (importing, functions)
- Pandas (DataFrames, filtering)
- The Densworld narrative (regions, characters)

---

## Part 1: Loading the Living Ledger

The Living Ledger isn't a CSV file—it's a special format called **JSONL** (JSON Lines), where each line is a separate event. We use a module called `ledger_lite` to query it.

First, let's set up our imports:

In [ ]:
import sys
import pandas as pd

# Tell Python where to find the ledger module
sys.path.insert(0, '../data')
import ledger_lite as ledger

print("Living Ledger module loaded!")
print("The Temporal Registry is now open.")

Let's see what's in the Ledger:

In [ ]:
# Get statistics about the event log
stats = ledger.stats()

print("LIVING LEDGER STATISTICS")
print("=" * 40)
print(f"Total events: {stats['total_events']:,}")
print(f"Unique actors (people): {stats['unique_actors']}")
print(f"Unique locations: {stats['unique_locations']}")

print("\nTop 10 event types:")
for event_type, count in sorted(stats['by_type'].items(), key=lambda x: -x[1])[:10]:
    print(f"  {event_type}: {count}")

*Hundreds of events. Dozens of actors. Multiple locations.*

*This is the history of Densworld—every expedition, every death, every discovery, every manuscript written.*

---

## Part 2: Querying Events by Type

The `ledger.query_events()` function lets us filter events. Let's find specific types of events.

### Finding Expeditions

In [ ]:
# Get all expedition launches
expeditions = ledger.query_events(event_type='expedition_launch')

print(f"Found {len(expeditions)} expedition launches")
print("\nFirst few expeditions:")
for e in expeditions[:5]:
    print(f"  {e['date']}: {e.get('details', {}).get('expedition_name', 'Unnamed')}")

### Converting to a DataFrame

We can convert events to a pandas DataFrame for familiar analysis:

In [ ]:
# Convert expeditions to DataFrame
exp_df = pd.DataFrame(expeditions)

print(f"Shape: {exp_df.shape}")
print(f"\nColumns: {list(exp_df.columns)}")

# Show key columns
exp_df[['event_id', 'date', 'location', 'actors']].head()

### Finding Disappearances

*Some expeditions don't end well. Let's see who has vanished in Densworld history.*

In [ ]:
# Get disappearances
disappearances = ledger.query_events(event_type='disappearance')

print(f"Found {len(disappearances)} disappearances")
print("\nWho vanished:")
for d in disappearances[:8]:
    actors = ', '.join(d.get('actors', ['unknown']))
    location = d.get('location', 'unknown')
    print(f"  {d['date']}: {actors} at {location}")

---

## Part 3: Querying by Actor

Remember the characters from earlier tutorials? Let's find events involving them.

### The Boss (Yeller Quarry)

In [ ]:
# Find events involving The Boss
boss_events = ledger.query_events(actor='the_boss')

print(f"Found {len(boss_events)} events involving The Boss")
print("\nThe Boss's story:")
for e in boss_events:
    print(f"  {e['date']}: {e['type'].replace('_', ' ')}")
    if e.get('notes'):
        # Show first 100 characters of notes
        print(f"    → {e['notes'][:100]}...")

*From legendary trapper to disappeared—the Ledger tells the whole story.*

### The Colonel (Tower of Mirado)

In [ ]:
# Find events involving The Colonel
colonel_events = ledger.query_events(actor='the_colonel')

print(f"Found {len(colonel_events)} events involving The Colonel")
for e in colonel_events:
    print(f"\n{e['event_id']} ({e['date']}): {e['type']}")
    if e.get('notes'):
        print(f"  {e['notes'][:150]}...")

*The siege that began in Year 820. Still ongoing.*

---

## Part 4: Querying by Location

We can find events by where they happened. The location filter does partial matching—so "yeller" finds anything containing "yeller".

In [ ]:
# Events at Yeller Quarry
yeller_events = ledger.query_events(location='yeller')

print(f"Events at Yeller Quarry: {len(yeller_events)}")

# What types of events happen there?
from collections import Counter
yeller_types = Counter(e['type'] for e in yeller_events)

print("\nWhat happens at Yeller Quarry:")
for event_type, count in yeller_types.most_common():
    print(f"  {event_type.replace('_', ' ')}: {count}")

In [ ]:
# Events at the Capital
capital_events = ledger.query_events(location='capital')

print(f"Events at the Capital: {len(capital_events)}")

capital_types = Counter(e['type'] for e in capital_events)
print("\nWhat happens at the Capital:")
for event_type, count in capital_types.most_common(5):
    print(f"  {event_type.replace('_', ' ')}: {count}")

*Notice the difference: Yeller Quarry has expeditions and disappearances. The Capital has manuscripts and debates. Each region has its own story.*

---

## Part 5: Querying by Date Range

We can filter events to a specific time period. Remember: Densworld uses 3-digit years (like 855, not 1855).

In [ ]:
# Events in the 850s (the decade of the Redmane Expedition)
decade_850s = ledger.query_events(date_start='850-01-01', date_end='859-12-31')

print(f"Events in the 850s: {len(decade_850s)}")
print("\nMajor events of this decade:")
for e in sorted(decade_850s, key=lambda x: x['date'])[:10]:
    print(f"  {e['date']}: {e['type'].replace('_', ' ')}")

---

## Part 6: Causal Chains

*The most powerful feature of the Living Ledger is **causation**. Each event can have a parent—the event that caused it.*

*When The Boss disappeared, an investigation was launched. The investigation exists because of the disappearance.*

Let's trace what happened after key events:

In [ ]:
# Find The Boss's disappearance
boss_disappearance = None
for d in disappearances:
    if 'the_boss' in d.get('actors', []):
        boss_disappearance = d
        break

if boss_disappearance:
    print(f"The Boss's disappearance: {boss_disappearance['event_id']}")
    print(f"Date: {boss_disappearance['date']}")
    print(f"Location: {boss_disappearance['location']}")
    
    # What happened as a result?
    consequences = ledger.get_descendants(boss_disappearance['event_id'])
    print(f"\nConsequences: {len(consequences)} events triggered")
    for c in consequences:
        print(f"  → {c['event_id']}: {c['type']} ({c['date']})")

### Tracing Backward: What Caused This?

We can also go the other direction—find what event caused another event.

In [ ]:
# Find events that have parents (were caused by something)
all_events = list(ledger.read_events())
events_with_causes = [e for e in all_events if e.get('parent')]

print(f"Events with known causes: {len(events_with_causes)}")

# Show a few examples
print("\nExamples of cause and effect:")
for e in events_with_causes[:5]:
    # Get the parent event
    parent = ledger.get_event(e['parent'])
    if parent:
        print(f"\n  {parent['type']} ({parent['date']})")
        print(f"    ↓ caused")
        print(f"  {e['type']} ({e['date']})")

---

## Part 7: Connecting to What You've Learned

Let's use the Ledger to explore the regions and characters from earlier tutorials.

In [ ]:
# Find the discovery of the yeller numbers (from Tutorial 3)
discoveries = ledger.query_events(event_type='discovery')

yeller_number_discovery = None
for d in discoveries:
    notes = d.get('notes', '').lower()
    details = str(d.get('details', {})).lower()
    if 'yeller' in notes or 'prime' in notes or 'yeller' in details:
        yeller_number_discovery = d
        break

if yeller_number_discovery:
    print("THE YELLER NUMBERS DISCOVERY")
    print("=" * 40)
    print(f"Event ID: {yeller_number_discovery['event_id']}")
    print(f"Date: {yeller_number_discovery['date']}")
    print(f"Location: {yeller_number_discovery['location']}")
    print(f"\nDetails:")
    print(f"  {yeller_number_discovery.get('notes', 'No notes')}")

In [ ]:
# Find Bagbu Olt's manuscript (from Tutorial 10)
bagbu_events = ledger.query_events(actor='bagbu_olt')

if bagbu_events:
    print(f"BAGBU OLT'S JOURNEY")
    print("=" * 40)
    for e in bagbu_events:
        print(f"\n{e['date']}: {e['type']}")
        if e.get('details'):
            details = e['details']
            if isinstance(details, dict):
                if 'title' in details:
                    print(f"  Title: {details['title']}")
                if 'quote' in details:
                    print(f"  Quote: \"{details['quote']}\"")

---

## Part 8: Event Distribution Over Time

Let's visualize when events happened across Densworld history.

In [ ]:
import matplotlib.pyplot as plt

# Count events by year
years = []
for e in all_events:
    date = e.get('date', '')
    if '-' in date:
        year = date.split('-')[0]
        if year.isdigit():
            years.append(int(year))

# Create histogram
plt.figure(figsize=(12, 5))
plt.hist(years, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Year')
plt.ylabel('Number of Events')
plt.title('When Did Events Happen in Densworld?')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nYear range: {min(years)} to {max(years)}")
print(f"Peak activity around: Year {max(set(years), key=years.count)}")

---

## Practice Exercises

### Exercise 1: Find All Deaths

Query all `death` events. Who died, when, and where?

In [ ]:
# Your code here:
# Hint: deaths = ledger.query_events(event_type='death')


### Exercise 2: Find Events at the Tower of Mirado

Use the location filter to find all events near Mirado. What types of events happened there?

In [ ]:
# Your code here:
# Hint: mirado_events = ledger.query_events(location='mirado')


### Exercise 3: Create a DataFrame of Manuscripts

Query all `manuscript_written` events and convert them to a DataFrame. Who wrote the most manuscripts?

In [ ]:
# Your code here:
# Hint:
# manuscripts = ledger.query_events(event_type='manuscript_written')
# df = pd.DataFrame(manuscripts)


### Exercise 4: Trace a Causal Chain

Pick an expedition launch event and trace its consequences using `ledger.get_descendants()`.

In [ ]:
# Your code here:
# Hint:
# expeditions = ledger.query_events(event_type='expedition_launch')
# first_exp = expeditions[0]
# consequences = ledger.get_descendants(first_exp['event_id'])


---

## Summary

You've learned to use the Living Ledger:

| Function | What It Does | Example |
|----------|-------------|----------|
| `ledger.stats()` | Get summary statistics | `stats = ledger.stats()` |
| `ledger.query_events()` | Filter events | `ledger.query_events(event_type='death')` |
| `ledger.query_events(actor=...)` | Filter by person | `ledger.query_events(actor='the_boss')` |
| `ledger.query_events(location=...)` | Filter by place | `ledger.query_events(location='yeller')` |
| `ledger.query_events(date_start=..., date_end=...)` | Filter by date range | `ledger.query_events(date_start='850-01-01')` |
| `ledger.get_event(id)` | Get single event | `ledger.get_event('EV-0855-002')` |
| `ledger.get_descendants(id)` | What happened next? | `ledger.get_descendants('EV-0855-001')` |
| `ledger.get_ancestors(id)` | What caused this? | `ledger.get_ancestors('EV-0855-003')` |

### Key Insight

The difference between `creatures.csv` and the Living Ledger:

- **creatures.csv** tells you *what exists* (static)
- **The Living Ledger** tells you *what happened* (temporal)

Both are valuable. Together, they tell the complete story of Densworld.

---

## Congratulations!

You've completed the Gateway to Densworld course.

You now know:
- **Python basics**: variables, lists, dictionaries, functions
- **Pandas**: loading data, filtering, grouping, merging
- **Visualization**: matplotlib charts and plots
- **The Living Ledger**: event-driven data with causation

And you know Densworld:
- The Capital and its Archives
- Yeller Quarry and its dangers
- The Dens and its dissolution
- The Tower of Mirado and the endless siege
- The edges: Northo and Dead River

---

## What's Next?

| If you want to... | Take this course |
|------------------|------------------|
| Master pandas | [Yeller Quarry Data Science](https://github.com/buildLittleWorlds/yeller-quarry-data-science) |
| Work with text | [Capital Archives NLP](https://github.com/buildLittleWorlds/capital-archives-nlp) |
| Learn ML math | [ML Math with Densworld](https://github.com/buildLittleWorlds/ml-math-with-densworld) |
| Explore philosophy | [Philosophy Series](https://github.com/buildLittleWorlds/philosophy-form-library) |
| Build AI apps | [Hugging Face Series](https://github.com/buildLittleWorlds/archivist-inference-engine) |

---

*"Every true journey begins in Yeller Quarry," Bagbu Olt wrote in his cart, somewhere between here and nowhere.*

*But some journeys begin with a single cell: Shift+Enter.*

*Yours began here. Where it goes next is up to you.*